# DataSet介绍

DataSet是Spark1.6引入的分布式数据集合，Spark2.0合并DataSet和DataFrame数据集合API，DataFrame变成DataSet的子集，DataFrame=DataSet[Row]。

DataSet是一个强类型，并且类型安全的数据容器，并且提供了结构化查询API和类似RDD一样的命令式API 。

# DataSet的创建

## 通过序列创建DataSet

In [ ]:
val ds = spark.range(1, 10, 2)

ds.show()

In [ ]:
val ds = spark.createDataset(0 to 10)

ds.show()

## 通过集合创建DataSet

In [ ]:
case class Person(name: String, age: Int)

val ds = spark.createDataset(Seq(Person("wux",10),Person("labs",20)))

ds.show()

## 通过RDD创建DataSet

In [ ]:
val rdd = sc.parallelize(List(1,2,3,4,5), 4)

val ds = rdd.toDS

ds.show()

In [ ]:
val rdd = sc.parallelize(List(Person("wux",10),Person("labs",20)), 4)

val ds = rdd.toDS

ds.show()

## 通过DataFrame创建DataSet

In [ ]:
case class Order (
val OrderNumber: String,
val OrderDate: String,
val ItemName: String,
val Quantity: Integer,
val ProductPrice: Double,
val TotalProducts: Integer
)

val df = spark.read.parquet("/mnt/databrickscontainer1/restaurant-1-orders.parquet")

val ds = df.as[Order]

ds.show()

# DataSet的底层

DataSet 最底层处理的是对象的序列化形式，通过查看 DataSet 生成的物理执行计划，也就是最终处理的RDD，就可以判定 DataSet 底层处理的是什么形式的数据。

In [ ]:
val rdd = ds.queryExecution.toRdd

dataset.queryExecution.toRdd 这个 API 可以看到 DataSet 底层执行的 RDD，这个 RDD 中的范型是 catalyst.InternalRow ，InternalRow 又称为 Catalyst Row ，是 DataSet 底层的数据结构，也就是说，无论 DataSet 的范型是什么，无论是 DataSet[Order] 还是其它的，其最底层进行处理的数据结构都是 InternalRow 。

所以， DataSet 的范型对象在执行之前，需要通过 Encoder 转换为 InternalRow，在输入之前，需要把 InternalRow 通过 Decoder 转换为范型对象。

* DataSet 是一个 Spark 组件，其底层还是 RDD 。
* DataSet 提供了访问对象中某个字段的能力，不用像 RDD 一样每次都要针对整个对象做操作。

# DataSet与DataFrame

## 相同的地方

### 相同的DSL访问

In [ ]:
df.printSchema()

ds.printSchema()

In [ ]:
df.describe().show()

ds.describe().show()

In [ ]:
df.select("OrderNumber","ItemName").show()

ds.select("OrderNumber","ItemName").show()

In [ ]:
df.filter("OrderNumber = 16117").show()

ds.filter("OrderNumber = 16118").show()

In [ ]:
df.sort(df("ProductPrice")).show()

ds.sort(ds("ProductPrice").desc).show()

### 相同的SQL访问

In [ ]:
df.createOrReplaceTempView("df")

spark.sql("select * from df where OrderNumber = 16117").show()

ds.createOrReplaceTempView("ds")

spark.sql("select * from ds where OrderNumber = 16118").show()

## 不同的地方

### 数据类型的不同

In [ ]:
val ds = spark.range(1, 10, 2)

val df = ds.toDF()

val df1 = df.first()

val ds1 = ds.first()

In [ ]:
case class Person(name: String, age: Int)

val ds = spark.createDataset(Seq(Person("wux",10),Person("labs",20)))

val df = ds.toDF()

val df1 = df.first()

val ds1 = ds.first()

In [ ]:
val rdd = sc.textFile("/mnt/databrickscontainer1/restaurant-1-orders.csv")

val ds = rdd.toDS

val df = ds.toDF()

val df1 = df.first()

val ds1 = ds.first()

In [ ]:
case class Order (
val OrderNumber: String,
val OrderDate: String,
val ItemName: String,
val Quantity: Integer,
val ProductPrice: Double,
val TotalProducts: Integer
)

val df = spark.read.parquet("/mnt/databrickscontainer1/restaurant-1-orders.parquet")

val ds = df.as[Order]

val df1 = df.first()

val ds1 = ds.first()

In [ ]:
df.map(x => x.getClass().getName()).show(3, false)

ds.map(x => x.getClass().getName()).show(3, false)

ds.toDF().map(x => x.getClass().getName()).show(3, false)

### 操作方式的不同

In [ ]:
df.printSchema()
df.map(x => x.getString(2)).show(5)

ds.printSchema()
ds.map(x => x.ItemName).show(5)

// df.map(x => x.ItemName).show(5)

### DataSet提供了编译时类型检查

## 相互转换

DataFrame和DataSet之间可以相互转换。


**DataFrame转为DataSet**

df.as[ElementType] 这样可以把DataFrame转化为DataSet。

**DataSet转为DataFrame**

ds.toDF() 这样可以把DataSet转化为DataFrame。

In [ ]:
case class Order (
val OrderNumber: String,
val OrderDate: String,
val ItemName: String,
val Quantity: Integer,
val ProductPrice: Double,
val TotalProducts: Integer
)

val df1 = spark.read.parquet("/mnt/databrickscontainer1/restaurant-1-orders.parquet")

val ds1 = df1.as[Order]

val df2 = ds1.toDF()